# COBRA model for (Geenen, 2012)

This notebook takes the original kinetic model in SBML format and saves a json model that allows for CBM calculations. 

In [1]:
import cobra
from cobra.solvers import get_solver_name
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import parsimonious
import pandas as pd

# Generate an updated JSON model for (Geenen,2012)

In [2]:
model = cobra.io.read_sbml_model('models/geenen_glutathione_original.xml')

Fix reaction reversibility

In [5]:
model.reactions.v_v1.lower_bound = 0 # met => SAM
model.reactions.v_v2.lower_bound = 0 # met => SAM
model.reactions.v_v3.lower_bound = 0 # SAM => SAH
model.reactions.v_v4.lower_bound = 0 # 
model.reactions.v_v6.lower_bound = 0 # hcy => met
model.reactions.v_v12.lower_bound = 0 # 2 GSH -> GSSG
model.reactions.v_v13.lower_bound = 0 # GSSG -> 2 GSH
model.reactions.v_v14.lower_bound = 0 # cGSSG => bGSSG
model.reactions.v_v15.lower_bound = 0 # cGSSG => bGSSG
model.reactions.v_v16.lower_bound = 0 # 
model.reactions.v_v17.lower_bound = 0 # 
model.reactions.v_v23.lower_bound = 0 # cTHF => cCH2THF
model.reactions.v_v25.lower_bound = 0 # oxo -> cglut
model.reactions.v_v33.lower_bound = 0 # cGSH + para => ASG
model.reactions.v_v34.lower_bound = 0 # cGSH + para => ASG
model.reactions.v_v40.lower_bound = 0 # bGSSG -> 2.0 bcys
model.reactions.v_v41.lower_bound = 0 # 

Modify the published SBML model and export it as a json for use with Escher
* Clearly mark exchange reactions with EX_ in the ID
* Exchange reactions should always carry negative flux to take up a metabolite
* Add paracetamol explictly to the model and FORCE its uptake

In [6]:
exchanges = [rxn for rxn in model.reactions if rxn.products == [] or rxn.reactants == []]
for r in exchanges:
    if 'EX_' not in r.id:
        r.id = 'EX_'+r.id
    model.repair()
    if list(r.metabolites.values())[0] == 1 :
        r.add_metabolites({list(r.metabolites.keys())[0]:-2})
        model.repair()
    r.lower_bound = -1; r.upper_bound = 1000
    print(r.id,r.reaction)
    
# Add paracetamol
para = Metabolite('para')
model.add_metabolites(para)
EX_para = Reaction('EX_para')
model.add_reaction(EX_para)
model.reactions.EX_para.add_metabolites({'para':-1})
model.reactions.EX_para.lower_bound = -1; model.reactions.EX_para.upper_bound = -1
model.reactions.v_v33.add_metabolites({'para':-1})
model.reactions.v_v34.add_metabolites({'para':-1})

# fix reaction 29 bug
# In the kinetic model bgly is considered to be a reservoir
# Upon importing into FBA this leads to this imbalanced reaction where cys = cysgly 
# We solve this by making this reaction produce gly in the cytosol 
# this is biochemically a bit strange but topologically does not matter 
model.reactions.v_v29.add_metabolites({'cgly':1})

cobra.io.save_json_model(model,'models/Geenen_cobra_model.json')
exchanges

EX_v_v18 cglut <=> 
EX_v_v20 cgly <=> 
EX_v_v22 cCH2THF <=> 
EX_v_v32 OPA <=> 
EX_v_v37 cysASG <=> 
EX_v_v38 oxo <=> 
EX_v_v39 met <=> 
EX_v_v41 bcys <=> 


[<Reaction EX_v_v18 at 0x10e013278>,
 <Reaction EX_v_v20 at 0x10e013470>,
 <Reaction EX_v_v22 at 0x10e0135c0>,
 <Reaction EX_v_v32 at 0x10e013cf8>,
 <Reaction EX_v_v37 at 0x10e0170b8>,
 <Reaction EX_v_v38 at 0x10e017128>,
 <Reaction EX_v_v39 at 0x10e0171d0>,
 <Reaction EX_v_v41 at 0x10e0173c8>]

Show all reactions and their bounds

In [8]:
df = pd.DataFrame.from_dict({'ID':[r.id for r in model.reactions],
                             'Reaction': [r.reaction for r in model.reactions],
                             'LB':[r.lower_bound for r in model.reactions],
                             'UB':[r.upper_bound for r in model.reactions] })
df[['ID','Reaction','LB','UB']]

,ID,Reaction,LB,UB
0,v_v1,met --> SAM,0,1000
1,v_v10,ccys + cglut <=> cglc,-1000,1000
2,v_v11,cglc + cgly <=> cGSH,-1000,1000
3,v_v12,2.0 cGSH --> cGSSG,0,1000
4,v_v13,cGSSG --> 2.0 cGSH,0,1000
5,v_v14,cGSSG --> bGSSG,0,1000
6,v_v15,cGSSG --> bGSSG,0,1000
7,v_v16,cGSH <=> bGSH,-1000,1000
8,v_v17,cGSH <=> bGSH,-1000,1000
9,EX_v_v18,cglut <=>,-1,1000
